In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
import os
import time
import datetime as dt
import packages.tools as tools

### Estimation/Forecasting Method:

### Note:
* In the paper, a 2-year rolling window was used for training the models, and the rest was out-of-sample data (specifics are a little unclear).
* They had 2 years and 7 months worth of data. On the other hand, our (Irish) dataset ranges only over 1 year and 6 months, so we will have to use a smaller 'training' window.

### Read in (pre-processed) dataset

In [2]:
path_dam_prices = "Datasets/DAMPrices.csv"
dam_prices = tools.preprocess_price_data(path_dam_prices)

In [3]:
pd.read_csv(path_dam_prices)

,AreaSet,AuctionDateTime,EURGBPRate,DeliveryPeriod,IntervalDuration,EURPrices
0,SEM-DA,2018-11-11 11:00:00,0.873668,2018-11-11 23:00:00,60,55.089
1,SEM-DA,2018-11-11 11:00:00,0.873668,2018-11-12 00:00:00,60,55.120
2,SEM-DA,2018-11-11 11:00:00,0.873668,2018-11-12 01:00:00,60,46.460
3,SEM-DA,2018-11-11 11:00:00,0.873668,2018-11-12 02:00:00,60,45.710
4,SEM-DA,2018-11-11 11:00:00,0.873668,2018-11-12 03:00:00,60,41.380
5,SEM-DA,2018-11-11 11:00:00,0.873668,2018-11-12 04:00:00,60,44.370
6,SEM-DA,2018-11-11 11:00:00,0.873668,2018-11-12 05:00:00,60,47.290
7,SEM-DA,2018-11-11 11:00:00,0.873668,2018-11-12 06:00:00,60,60.440
8,SEM-DA,2018-11-11 11:00:00,0.873668,2018-11-12 07:00:00,60,63.979
9,SEM-DA,2018-11-11 11:00:00,0.873668,2018-11-12 08:00:00,60,67.190


In [4]:
dam_prices.head()

,AuctionDateTime,EURPrices
DeliveryPeriod,,
2018-11-12 00:00:00,2018-11-11 11:00:00,55.089
2018-11-12 01:00:00,2018-11-11 11:00:00,55.120
2018-11-12 02:00:00,2018-11-11 11:00:00,46.460
2018-11-12 03:00:00,2018-11-11 11:00:00,45.710
2018-11-12 04:00:00,2018-11-11 11:00:00,41.380


In [61]:
"""
Parameters:
    data: pd.DataFrame
    target: datetime
    day_lag: int
        - Number of days to go back in order to make forecast, e.g. day_lag=7 indicates a weekly persistent model,
        day_lag=1 indicates a daily persistent model, etc.
"""

def naive(data, target, day_lag):
    if "AuctionDateTime" in data.columns:
        data.drop("AuctionDateTime", axis=1, inplace=True)
    
    # Make forecasts (depending on day_lag)
    delivery_day = target - dt.timedelta(days=day_lag)
    forecast_df = data.loc[data.index.date == delivery_day.date()]
        
    # Create and prepare forecasts dataframe
    forecast_df.index = pd.date_range(target, periods=24, freq="H")
    forecast_df.index.name = "DeliveryPeriod"
    
    return(forecast_df)

In [63]:
test_df = naive(data=dam_prices, target=dt.datetime.strptime("2018-11-13", "%Y-%m-%d"), day_lag=1)
test_df

,EURPrices
DeliveryPeriod,
2018-11-13 00:00:00,55.089
2018-11-13 01:00:00,55.120
2018-11-13 02:00:00,46.460
2018-11-13 03:00:00,45.710
2018-11-13 04:00:00,41.380
2018-11-13 05:00:00,44.370
2018-11-13 06:00:00,47.290
2018-11-13 07:00:00,60.440
2018-11-13 08:00:00,63.979


In [8]:
naive_params = dict(day_lag=2)
walk_forward_validation(method_function=naive, parameters=naive_params, data=dam_prices,
                        starting_window_size=2, moving_window=True, start=None, end=None)

EURPrices_forecast  EURPrices_data residual  \
DeliveryDay TimeStepID                                               
2018-11-14  0                      55.089          48.010   -7.079   
            1                       55.12          35.000   -20.12   
            2                       46.46           6.000   -40.46   
            3                       45.71           1.030   -44.68   
            4                       41.38           0.000   -41.38   
            5                       44.37           0.000   -44.37   
            6                       47.29           0.000   -47.29   
            7                       60.44          22.370   -38.07   
            8                      63.979          47.180  -16.799   
            9                       67.19          61.050    -6.14   
            10                     72.037          65.200   -6.837   
            11                     69.097          62.224   -6.873   
            12                     67.765          61.120   -6.645   
            13                     65.283          61.996   -3.287   
            14                     67.212          64.000   -3.212   
            15                     68.033          61.120   -6.913   
            16                     72.188          70.000   -2.188   
            17                      89.41          80.252   -9.158   
            18                     106.64         100.200    -6.44   
            19                     89.932          85.170   -4.762   
            20                     74.533          74.299   -0.234   
            21                     64.163          65.716    1.553   
            22                     64.195          62.000   -2.195   
            23                      59.35          50.000    -9.35   

                       absolute_error squared_error  
DeliveryDay TimeStepID                               
2018-11-14  0                   7.079       50.1122  
            1                   20.12       404.814  
            2                   40.46       1637.01  
            3                   44.68        1996.3  
            4                   41.38        1712.3  
            5                   44.37        1968.7  
            6                   47.29       2236.34  
            7                   38.07       1449.32  
            8                  16.799       282.206  
            9                    6.14       37.6996  
            10                  6.837       46.7446  
            11                  6.873       47.2381  
            12                  6.645        44.156  
            13                  3.287       10.8044  
            14                  3.212       10.3169  
            15                  6.913       47.7896  
            16                  2.188       4.78734  
            17                  9.158        83.869  
            18                   6.44       41.4736  
            19                  4.762       22.6766  
            20                  0.234      0.054756  
            21                  1.553       2.41181  
            22                  2.195       4.81802  
            23                   9.35       87.4225